In [1]:
CPU = False

import os
if CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import cv2
import keras
import keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import glob

from tqdm import tqdm
from keras.callbacks import ModelCheckpoint

from ssd_model import SSD512 as Model
from ssd_utils import PriorUtil
from ssd_data import InputGenerator
from ssd_data import preprocess
from ssd_training import SSDLoss
from ssd_metric import evaluate_results
from utils.model import load_weights
from utils.training import Logger, LearningRateDecay

Using TensorFlow backend.


### Data and Model

In [ ]:
data_path = './data/RoboTT/'
#data_path = './data/RoboTT_extra/'

from data_robott import GTUtility
gt_util = GTUtility(data_path)
gt_util_train, gt_util_val = gt_util.split(0.8)
print(gt_util)

In [ ]:
experiment = 'robott'

model = Model(num_classes=gt_util.num_classes)

prior_util = PriorUtil(model)

In [ ]:
# initialize with SSD model
load_weights(model, './models/ssd512_voc_weights_fixed.hdf5')

freeze = ['conv1_1', 'conv1_2',
          'conv2_1', 'conv2_2',
          'conv3_1', 'conv3_2', 'conv3_3',
          #'conv4_1', 'conv4_2', 'conv4_3',
          #'conv5_1', 'conv5_2', 'conv5_3',
         ]
for layer in model.layers:
    layer.trainable = not layer.name in freeze

initial_epoch = 0

In [ ]:
# continue training
#load_weights(model, './checkpoints/201708072158_ssd_voc/weights.400.h5')
#load_weights(model, './checkpoints/201709202151_robott/weights.067.h5'); initial_epoch = 68
#load_weights(model, './checkpoints/201709231742_robott/weights.058.h5'); initial_epoch = 59
load_weights(model, './checkpoints/201710052244_robott/weights.118.h5'); initial_epoch = 119

In [ ]:
epochs = 500
batch_size = 24

options = {
    'vflip_prob': 0.0, 
    'hflip_prob': 0.0, 
    'do_crop': True, 
    'crop_area_range': [0.6, 1.0]
}
gen_train = InputGenerator(gt_util_train, prior_util, batch_size, model.image_size, augmentation=True, **options)
gen_val = InputGenerator(gt_util_val, prior_util, batch_size, model.image_size, augmentation=True, **options)

### Training

In [ ]:
checkdir = './checkpoints/' + time.strftime('%Y%m%d%H%M') + '_' + experiment
if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())

#optim = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optim = keras.optimizers.SGD(lr=1e-3, momentum=0.9, decay=0, nesterov=True)

# weight decay
regularizer = keras.regularizers.l2(5e-4) # None if disabled
for l in model.layers:
    if l.__class__.__name__.startswith('Conv'):
        l.kernel_regularizer = regularizer

loss = SSDLoss(alpha=1.0, neg_pos_ratio=3.0)

model.compile(optimizer=optim,
              loss=loss.compute,
              metrics=loss.metrics)

callbacks = [
    ModelCheckpoint(checkdir+'/weights.{epoch:03d}.h5', verbose=1, save_weights_only=True),
    Logger(checkdir),
    LearningRateDecay()
]

history = model.fit_generator(gen_train.generate(), #generator, 
                              gen_train.num_batches, #steps_per_epoch, 
                              epochs=epochs, 
                              verbose=1, 
                              callbacks=callbacks, 
                              validation_data=gen_val.generate(), 
                              validation_steps=gen_val.num_batches, 
                              class_weight=None, 
                              #max_queue_size=10, 
                              workers=1, 
                              #use_multiprocessing=False, 
                              initial_epoch=initial_epoch)

### Example results

In [ ]:
#gt_util = gt_util_train
gt_util = gt_util_val
#np.random.seed(1337)
idxs = np.random.randint(0, gt_util.num_samples, 16)
h, w = model.image_size

data = []
inputs = []
images = []
for i in idxs:
    img_path = os.path.join(gt_util.image_path, gt_util.image_names[i])
    img = cv2.imread(img_path)
    gt = gt_util.data[i]
    
    # random cropping
    for _ in range(32): # retries if lose ground truth
        tmp_img, tmp_gt = gen_val.random_sized_crop(img, gt)
        if len(tmp_gt) > 0: break
    if len(tmp_gt) == 0:
        print('no gt', gt_util.image_names[i])
        continue
    else:
        img = tmp_img
        gt = tmp_gt
        
    inputs.append(preprocess(img, model.image_size))
    img = cv2.resize(img, (w,h), cv2.INTER_LINEAR)
    img = img[:, :, (2,1,0)] # BGR to RGB
    img = img / 256.
    images.append(img)
    data.append(gt)
inputs = np.asarray(inputs)

preds = model.predict(inputs, batch_size=1, verbose=1)

checkdir = './checkpoints/201710052244_robott'
for fl in glob.glob('%s/result_*' % (checkdir,)):
    os.remove(fl)

for i in range(len(preds)):
    res = prior_util.decode(preds[i], confidence_threshold=0.2, keep_top_k=3)
    if len(data[i]) > 0:
        plt.figure(figsize=[10]*2)
        plt.imshow(images[i])
        prior_util.plot_results(res, classes=gt_util.classes, show_labels=True, gt_data=data[i])
        plt.savefig('%s/result_%03d.jpg' % (checkdir, i))
        plt.show()

### Evaluation

In [ ]:
#gt_util = gt_util_train
gt_util = gt_util_val

batch_size = 32

gt_all = []
dt_all = []
in_batch = []
gt_batch = []

for i in tqdm(range(gt_util.num_samples)):
    img_path = os.path.join(gt_util.image_path, gt_util.image_names[i])
    img = cv2.imread(img_path)
    gt = gt_util.data[i]
    
    if True:
        # random cropping
        for _ in range(32): # retries if lose ground truth
            tmp_img, tmp_gt = gen_val.random_sized_crop(img, gt)
            if len(tmp_gt) > 0: break
        if len(tmp_gt) == 0:
            print('no gt', gt_util.image_names[i])
            #continue
        else:
            img = tmp_img
            gt = tmp_gt
    
    in_batch.append(preprocess(img, model.image_size))
    gt_batch.append(gt)
    
    # images for plot
    #img_h, img_w = model.image_size
    #img = cv2.resize(img, (img_w, img_h), cv2.INTER_LINEAR)
    #img = img[:, :, (2,1,0)] # BGR to RGB
    #img = img / 256.
    #images.append(img)
    
    if len(in_batch) == batch_size or i == gt_util.num_samples-1:
        preds = model.predict(np.asarray(in_batch), batch_size=batch_size, verbose=0)
        for j in range(len(preds)):
            dt = prior_util.decode(preds[j], confidence_threshold=0.01, keep_top_k=400, fast_nms=False)
            gt_all.append(gt_batch[j])
            dt_all.append(dt)
        in_batch = []
        gt_batch = []


In [ ]:
evaluate_results(gt_all, dt_all, gt_util, iou_thresh=0.1, max_dets=1)

### Detections speed

In [ ]:
%%timeit
preds = model.predict(inputs[None,0,:,:,:], batch_size=1, verbose=0)
res = prior_util.decode(preds[0], confidence_threshold=0.2, keep_top_k=1)

### Test images

In [ ]:
img_dir = './data/images/test_images_robott/'

inputs = []
images = []
for img_path in glob.glob(img_dir+'/20171019_*'):
    img = cv2.imread(img_path)
    inputs.append(preprocess(img, model.image_size))
    h, w = model.image_size
    img = cv2.resize(img, (w,h), cv2.INTER_LINEAR)
    img = img[:, :, (2,1,0)] # BGR to RGB
    img = img / 256.
    images.append(img)
inputs = np.asarray(inputs)
    
preds = model.predict(inputs, batch_size=1, verbose=1)

checkdir = img_dir
for fl in glob.glob('%s/result_*' % (checkdir,)):
    os.remove(fl)

for i in range(len(preds)):
    res = prior_util.decode(preds[i], confidence_threshold=0.2, keep_top_k=1)
    
    plt.figure(figsize=[10]*2)
    plt.imshow(images[i])
    print(len(res))
    prior_util.plot_results(res, classes=gt_util.classes, show_labels=True)
    plt.savefig('%s/result_%03d.jpg' % (checkdir, i))
    plt.show()